### --- Day 6: Chronal Coordinates ---
- Get the Points (call them destinations)
- From origin {0,0}, sort list into an ordereddict
- Define grid from the lowest to highest distinations from origin
- Create an object to represent each node that contains a destination flag and an array of nearest neighbors
- Iterate entire grid finding nearest destination to each point using Manhatten distance formula (if x=(a,b) and y=(c,d), then |a−c|+|b−d|)
- iterate entire grid, for each position add all unique nearest neighbors to a hashmap.
- display largest value

In [1]:
class Coordinate():
    def __init__(self, line):
        x = line.split(",")[0].strip()
        y = line.split(",")[1].strip()
        self.x = int(x)
        self.y = int(y)
    def __str__(self):
        return "[{0},{1}]".format(self.x,self.y)
    def __repr__(self):
        return self.__str__()

val1 = Coordinate("0,0")
val2 = Coordinate("1,1")
print(val1)
print(val2)

[0,0]
[1,1]


In [2]:
coords = open("test_input.txt", "r").readlines()
print(coords)

['1, 1\n', '1, 6\n', '8, 3\n', '3, 4\n', '5, 5\n', '8, 9']


In [3]:
def getManhattenDistanceBetween(val1,val2):
    if (not isinstance(val1,Coordinate) and not isinstance(val2,Coordinate)):
        print("Invalid input")
        return 
    #if debug: print(str(val1) + " & " + str(val2))
    return abs(val1.x-val2.x) + abs(val1.y-val2.y)

getManhattenDistanceBetween(Coordinate("0,0"),Coordinate("1, 1\n"))

2

In [4]:
def name_destinations(coords):
    val = 65 
    results = []
    for coord in coords:
        c = Coordinate(coord)
        result = (chr(val), c)
        results.append(result)
        val += 1
        
    return results

destinations = name_destinations(coords)
print(destinations)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]


In [5]:
for d in destinations:
    print(d)
    
x_max = max(p[1].x for p in destinations)
y_max = max(p[1].y for p in destinations)
print("x_max: " + str(x_max))
print("y_max: " + str(y_max))

('A', [1,1])
('B', [1,6])
('C', [8,3])
('D', [3,4])
('E', [5,5])
('F', [8,9])
x_max: 8
y_max: 9


In [6]:
from pandas import * #! pip3 install pandas

x_max = max(p[1].x for p in destinations) + 1 #zero based
y_max = max(p[1].y for p in destinations) + 1 #zero based

dest_map = DataFrame([[0 for y in range(x_max)] for i in range(y_max)])

for name, coord in destinations:
    dest_map.loc[coord.y,coord.x] =  name     #faster, treats as one call

print(destinations)    
print(dest_map)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]
   0  1  2  3  4  5  6  7  8
0  0  0  0  0  0  0  0  0  0
1  0  A  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0  0  0
3  0  0  0  0  0  0  0  0  C
4  0  0  0  D  0  0  0  0  0
5  0  0  0  0  0  E  0  0  0
6  0  B  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0  0
8  0  0  0  0  0  0  0  0  0
9  0  0  0  0  0  0  0  0  F


In [7]:
debug = True
def closest_match(position):
    distance = 1000
    match_destination = None
    for destination in destinations:
        tmp = getManhattenDistanceBetween(position,destination[1])
        
        if debug: print("{2}::{0}->{1}".format(destination[1],tmp,destination[0]))
        
        if not match_destination is None:
            if tmp<distance:                     #least distance
                distance = tmp
                match_destination = destination  #multiple match
            elif tmp==distance:
                match_destination = "."
            else:
                next                             #stop looking
            
        else:
            match_destination = destination[0]
            distance = tmp
    return match_destination 
        
dest = closest_match(Coordinate("1,4"))   
print("Closest destination to [1,4]: " + str(dest))

A::[1,1]->3
B::[1,6]->2
C::[8,3]->8
D::[3,4]->2
E::[5,5]->5
F::[8,9]->12
Closest destination to [1,4]: .


In [8]:
debug = False

def buildMap(dest_map):
    for row_index, row in dest_map.iterrows():
        for col_index, val in enumerate(row):
            if val == 0:
                addr = "{0},{1}".format(col_index,row_index)
                dest = closest_match(Coordinate(addr)) 
                if debug: print("closest match to {0} is {1}".format(addr,dest))
                dest_map.loc[row_index,col_index] = dest[0].lower()
            else:
                next
        print("row {0} done".format(row_index))

buildMap(dest_map)
print(dest_map)

row 0 done
row 1 done
row 2 done
row 3 done
row 4 done
row 5 done
row 6 done
row 7 done
row 8 done
row 9 done
   0  1  2  3  4  5  6  7  8
0  a  a  a  a  a  .  c  c  c
1  a  A  a  a  a  .  c  c  c
2  a  a  a  d  d  e  c  c  c
3  a  a  d  d  d  e  c  c  C
4  .  .  d  D  d  e  e  c  c
5  b  b  .  d  e  E  e  e  c
6  b  B  b  .  e  e  e  e  .
7  b  b  b  .  e  e  e  f  f
8  b  b  b  .  e  e  f  f  f
9  b  b  b  .  f  f  f  f  F


In [16]:
def mapResults(input):
    dict = {}
    for i, row in dest_map.iterrows():
        for j, val in row.iteritems():
            if not val == '.':
                if val.lower() in dict:
                    dict[val.lower()] += 1
                else:
                    dict[val.lower()] = 1
    return dict
        
print(mapResults(dict))    

def getMax(dict):
    max_i = 0
    max_k = ''
    for key, value in dict.items():
        if dict[key]>max_i:
            max_i = value
            max_k = key
    return max_k,max_i

result = getMax(dict)
print("Max is {0}: {1}".format(result[0],result[1]))


{'a': 15, 'c': 15, 'd': 9, 'e': 17, 'b': 14, 'f': 10}


TypeError: descriptor 'items' of 'dict' object needs an argument

In [ ]:
coords = open("--6.txt", "r").readlines()
destinations = name_destinations(coords)

x_max = max(p[1].x for p in destinations) + 1 #zero based
y_max = max(p[1].y for p in destinations) + 1 #zero based

dest_map = DataFrame([[0 for y in range(x_max)] for i in range(y_max)])
buildMap(dest_map)
print("- Built Map")
results = mapResults(dest_map)
print("- Mapping Results")
result = getMap(dict)
print("Max is {0}: {1}".format(result[0],result[1]))
